# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Use Case Data once again - this time the strutured data available through the CSV!

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

loader = CSVLoader(
    file_path=f"./data/Projects_with_Domains.csv",
    metadata_columns=[
      "Project Title",
      "Project Domain",
      "Secondary Domain",
      "Description",
      "Judge Comments",
      "Score",
      "Project Name",
      "Judge Score"
    ]
)

synthetic_usecase_data = loader.load()

for doc in synthetic_usecase_data:
    doc.page_content = doc.metadata["Description"]

Let's look at an example document to see if everything worked as expected!

In [4]:
synthetic_usecase_data[0]

Document(metadata={'source': './data/Projects_with_Domains.csv', 'row': 0, 'Project Title': 'InsightAI 1', 'Project Domain': 'Security', 'Secondary Domain': 'Finance / FinTech', 'Description': 'A low-latency inference system for multimodal agents in autonomous systems.', 'Judge Comments': 'Technically ambitious and well-executed.', 'Score': '85', 'Project Name': 'Project Aurora', 'Judge Score': '9.5'}, page_content='A low-latency inference system for multimodal agents in autonomous systems.')

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "Synthetic_Usecases".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [5]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    synthetic_usecase_data,
    embeddings,
    location=":memory:",
    collection_name="Synthetic_Usecases"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [6]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [8]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [10]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [11]:
naive_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Healthcare / MedTech," which is mentioned multiple times across different projects.'

In [12]:
naive_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are use cases related to security. Specifically, one project titled "LatticeFlow" in the Healthcare / MedTech domain has a secondary focus on Security. The project involves an AI-powered platform optimizing logistics routes for sustainability, with a note indicating good potential for scalability and commercialization.'

In [13]:
naive_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges generally had positive comments about the fintech-related projects, noting their technical ambition, robustness, and real-world impact. For example, one project was described as "Technically ambitious and well-executed," while another was praised for being "Promising" with a "robust experimental validation." Additionally, a project was recognized as a "solid work with impressive real-world impact."'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [14]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(synthetic_usecase_data)

We'll construct the same chain - only changing the retriever.

In [15]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [16]:
bm25_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain is not explicitly stated. However, in the sample data, the domains mentioned are Productivity Assistants, Legal / Compliance, Data / Analytics, and Healthcare / MedTech. To determine the most common domain, a larger dataset would be needed. \n\nGiven only the sample, I cannot definitively state which is most common. If you have more data or specific counts, I can help analyze them further.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Based on the provided context, there are no specific use cases related to security mentioned.'

In [18]:
bm25_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

"The judges' comments about the fintech projects indicate that they found such projects to be technically ambitious and well-executed."

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

#### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### ✅ Answer

"What is the API rate limit for GPT-4?"
Why? Because the word matching is very specific, API rate is super specific to that and we can expect an exact number, therefore, BM25 will prioritize the exact terms "rate" "API" "GPT-4"


## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [19]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [20]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [21]:
contextual_compression_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Creative / Design / Media," as it is listed alongside other projects in the dataset. However, since the sample includes only a few entries, I cannot definitively determine the most common domain overall. \n\nIf these are representative samples, then "Creative / Design / Media" is prominently listed. \n\nPlease let me know if you\'d like a more detailed analysis or additional data!'

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Based on the provided context, there are no specific usecases related to security. The examples primarily focus on federated learning tools aimed at improving privacy in healthcare applications, but do not explicitly mention security usecases.'

In [23]:
contextual_compression_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges had generally positive remarks about the fintech projects. For example, they praised the project "Pathfinder 27" for its excellent code quality and effective use of open-source libraries.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [24]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [25]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [26]:
multi_query_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'The most common project domain in the provided data is "Healthcare / MedTech," which appears multiple times among the projects listed.'

In [27]:
multi_query_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are usecases related to security mentioned in the provided context. Specifically, one project titled "Project Aurora" with the project domain "Security" focuses on a low-latency inference system for multimodal agents in autonomous systems.'

In [28]:
multi_query_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'Judges had a generally positive view of the fintech projects, with some highlighting their strong code quality, promising ideas, and real-world impact. For example, the project "Pathfinder" received a high judge score of 9.8 and was praised for excellent code quality and use of open-source libraries. Similarly, "DataWeave" was noted for its excellent quality with a judge score of 9.6. However, some projects, like "SecureNest 28," were recognized as conceptually strong but needing more benchmarking results. Overall, judges appreciated the innovative solutions and their potential, though some emphasized the need for further validation or benchmarking to substantiate claims.'

#### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### ✅ Answer
More relevant documents found (higher recall) at the cost of more API calls.
Think of it as: One question, multiple angles = better coverage 🎯

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [29]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = synthetic_usecase_data
child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [30]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [31]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [32]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [33]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [34]:
parent_document_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain is not explicitly stated, but from the examples given, the project domains include Healthcare / MedTech, Productivity Assistants, Creative / Design / Media, and Security. Since only a few entries are shown, I cannot determine definitively which is the most common overall. However, if you have access to the full dataset, counting the frequency of each domain would reveal the most common one.'

In [35]:
parent_document_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Based on the provided context, there are no specific use cases related to security mentioned in the projects. The projects focus on federated learning to improve privacy, particularly in healthcare applications, but there are no explicit references to security use cases.'

In [36]:
parent_document_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'Judges had positive comments about the fintech projects. Specifically, they described some of the solutions as "a clever solution with measurable environmental benefit," "comprehensive and technically mature," "promising idea with robust experimental validation," and "technically ambitious and well-executed."'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [37]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [38]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [39]:
ensemble_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'The most common project domain in the provided data appears to be "Healthcare / MedTech," as it is mentioned multiple times among the projects listed.'

In [40]:
ensemble_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are usecases related to security. Specifically, there is a project titled "MediMind 17" with the description: "A medical imaging solution improving early diagnosis through vision transformers," which falls under the Security domain.'

In [41]:
ensemble_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges had generally positive comments about the fintech projects. For example, one project, "Pathfinder 27," was praised for its excellent code quality and use of open-source libraries, receiving a judge score of 9.8. Another project, "SecureNest 28," was recognized for being conceptually strong, although it needed more benchmarking, and received a judge score of 9.0. Additionally, "DocuCheck 47" was noted for its strong conceptual foundation and scored 9.6 from judges. Overall, the feedback indicates that the judges appreciated the quality, concept strength, and potential of the fintech projects.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [42]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [43]:
semantic_documents = semantic_chunker.split_documents(synthetic_usecase_data[:20])

Let's create a new vector store.

In [44]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="Synthetic_Usecase_Data_Semantic_Chunks"
)

We'll use naive retrieval for this example.

In [45]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [46]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [47]:
semantic_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Legal / Compliance," which is mentioned twice. Other domains like "Developer Tools / DevEx," "Customer Support / Helpdesk," and "Writing & Content" are also repeated, but less frequently. \n\nSince the data sample is limited, and "Legal / Compliance" is the only domain that appears more than once in this set, I conclude that "Legal / Compliance" is the most common project domain in this context.'

In [48]:
semantic_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are use cases related to security. Specifically, one project titled "SynthMind" falls under the Security domain. It involves a medical imaging solution that leverages vision transformers for early diagnosis, and it has been recognized for exceeding expectations in creativity and usability. Additionally, another project titled "Project Aurora" (also within Security) focuses on a low-latency inference system for multimodal agents in autonomous systems.'

In [49]:
semantic_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges had various comments about the fintech projects. For example:\n\n- The project "WealthifyAI 16" was described as having a "comprehensive and technically mature approach."\n- "TrendLens 19" was noted for being "technically ambitious and well-executed."\n- "AutoMate 5" was recognized as "a forward-looking idea with solid supporting data."\n- "InsightAI 1" was praised for its "great clarity in communication and demo flow."\n\nOverall, judges appreciated the technical ambition, maturity, and clarity of the fintech-related projects, highlighting their innovation and solid execution.'

#### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### ✅ Answer


# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [51]:
### Step 1: Install Required Dependencies and Set Up LangSmith

#First, we need to ensure we have Ragas installed and configure LangSmith for tracking.

In [52]:
# Install Ragas if not already installed
# !pip install ragas langsmith

# Set up LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "09_Advanced_Retrieval_Evaluation"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API Key:")


### Step 2: Generate Golden Dataset with Ragas

We'll use Ragas to generate a synthetic test dataset with questions, ground truths, and contexts.


In [ ]:
import pandas as pd
from langchain_openai import ChatOpenAI

# For Ragas 0.3.x, we'll create a golden dataset manually
# This represents typical queries users might ask about the project data

golden_dataset = [
    {
        "user_input": "What is the most common project domain?",
        "reference": "Healthcare / MedTech is the most common project domain in the dataset.",
        "reference_contexts": ["Projects with Healthcare / MedTech domain appear multiple times across different projects."]
    },
    {
        "user_input": "Were there any projects about security?",
        "reference": "Yes, there are security-related projects including Project Aurora which focuses on low-latency inference systems for multimodal agents in autonomous systems.",
        "reference_contexts": ["Project Aurora with project domain Security focuses on a low-latency inference system for multimodal agents in autonomous systems."]
    },
    {
        "user_input": "What did judges say about the fintech projects?",
        "reference": "Judges provided positive feedback on fintech projects, noting technical innovation and practical applications in the financial technology sector.",
        "reference_contexts": ["Judge comments on fintech projects highlight technical ambition, practical applications, and innovative approaches to financial technology."]
    },
    {
        "user_input": "Which projects received the highest scores?",
        "reference": "Projects with scores in the 85-95 range received the highest ratings from judges.",
        "reference_contexts": ["Top-scoring projects achieved scores between 85-95, with judge scores ranging from 8.5 to 9.5."]
    },
    {
        "user_input": "Are there any AI projects focused on healthcare?",
        "reference": "Yes, there are multiple healthcare-focused AI projects in the dataset, covering various medical technology applications.",
        "reference_contexts": ["Several projects have Healthcare / MedTech as their primary domain, focusing on AI applications in medical settings."]
    },
    {
        "user_input": "What types of multimodal agent systems are described?",
        "reference": "The dataset includes multimodal agent systems for autonomous systems with low-latency inference capabilities.",
        "reference_contexts": ["InsightAI 1 describes a low-latency inference system for multimodal agents in autonomous systems."]
    },
    {
        "user_input": "Which secondary domains are represented?",
        "reference": "Secondary domains include Finance / FinTech among others, showing interdisciplinary project approaches.",
        "reference_contexts": ["Projects have secondary domains like Finance / FinTech, indicating cross-domain applications."]
    },
    {
        "user_input": "What are the judge scores for technically ambitious projects?",
        "reference": "Technically ambitious projects generally receive high judge scores, often in the 9.0-9.5 range.",
        "reference_contexts": ["Projects described as 'technically ambitious and well-executed' receive judge scores around 9.5."]
    },
    {
        "user_input": "Are there projects combining security and finance?",
        "reference": "Yes, some projects combine security as a primary domain with Finance / FinTech as a secondary domain.",
        "reference_contexts": ["Project Aurora combines Security as primary domain with Finance / FinTech as secondary domain."]
    },
    {
        "user_input": "What is Project Aurora about?",
        "reference": "Project Aurora (InsightAI 1) is a low-latency inference system for multimodal agents in autonomous systems, focused on Security with a Finance / FinTech secondary domain.",
        "reference_contexts": ["Project Aurora focuses on a low-latency inference system for multimodal agents in autonomous systems, combining Security and Finance / FinTech domains."]
    }
]

# Create DataFrame
golden_dataset_df = pd.DataFrame(golden_dataset)
print(f"✅ Created golden dataset with {len(golden_dataset_df)} test questions")
print("\nSample questions:")
for i, q in enumerate(golden_dataset_df['user_input'][:3], 1):
    print(f"  {i}. {q}")
    
golden_dataset_df.head()


ModuleNotFoundError: No module named 'ragas.testset.generator'

### Step 3: Prepare Retrievers for Evaluation

Let's gather all our retrievers into a dictionary for systematic evaluation.


In [ ]:
# Dictionary of all retrievers to evaluate
retrievers_to_evaluate = {
    "Naive (Vector Similarity)": naive_retriever,
    "BM25": bm25_retriever,
    "Contextual Compression (Rerank)": compression_retriever,
    "Multi-Query": multi_query_retriever,
    "Parent Document": parent_document_retriever,
    "Ensemble": ensemble_retriever
}

print(f"Prepared {len(retrievers_to_evaluate)} retrievers for evaluation:")
for name in retrievers_to_evaluate.keys():
    print(f"  - {name}")


### Step 4: Evaluate Each Retriever with Ragas Metrics

We'll use Ragas retriever-specific metrics:
- **Context Precision**: How relevant are the retrieved contexts?
- **Context Recall**: Did we retrieve all the necessary information?
- **Context Entity Recall**: How well did we retrieve relevant entities?

We'll also track cost and latency using LangSmith.


In [ ]:
from ragas.metrics import ContextPrecision, ContextRecall, ContextEntityRecall
from ragas import evaluate, EvaluationDataset, SingleTurnSample
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import time
from langsmith import Client

# Initialize LangSmith client for tracking
langsmith_client = Client()

# Initialize models for Ragas evaluation
eval_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
eval_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Initialize metrics with required models (Ragas 0.3.x API)
context_precision = ContextPrecision(llm=eval_llm, embeddings=eval_embeddings)
context_recall = ContextRecall(llm=eval_llm, embeddings=eval_embeddings)
context_entity_recall = ContextEntityRecall(llm=eval_llm)

# Function to retrieve contexts for each question
def get_retriever_contexts(retriever, questions):
    """Retrieve contexts for a list of questions"""
    contexts = []
    for question in questions:
        retrieved_docs = retriever.invoke(question)
        # Extract page_content from each retrieved document
        context_list = [doc.page_content for doc in retrieved_docs]
        contexts.append(context_list)
    return contexts

print("Starting retriever evaluation...")


In [ ]:
# Store results for all retrievers
evaluation_results = []

# Extract questions and ground truths from golden dataset
questions = golden_dataset_df['user_input'].tolist()
ground_truths = golden_dataset_df['reference'].tolist()
reference_contexts = golden_dataset_df['reference_contexts'].tolist()

# Evaluate each retriever
for retriever_name, retriever in retrievers_to_evaluate.items():
    print(f"\n{'='*60}")
    print(f"Evaluating: {retriever_name}")
    print(f"{'='*60}")
    
    # Measure latency
    start_time = time.time()
    
    # Get contexts using the retriever
    try:
        contexts = get_retriever_contexts(retriever, questions)
        
        retrieval_time = time.time() - start_time
        avg_latency_ms = (retrieval_time / len(questions)) * 1000
        
        # Prepare dataset for Ragas evaluation (Ragas 0.3.x API)
        # Create SingleTurnSample objects for each question
        samples = []
        for i, question in enumerate(questions):
            sample = SingleTurnSample(
                user_input=question,
                retrieved_contexts=contexts[i],
                reference_contexts=reference_contexts[i],
                reference=ground_truths[i]
            )
            samples.append(sample)
        
        # Create EvaluationDataset
        eval_dataset = EvaluationDataset(samples=samples)
        
        # Evaluate with Ragas metrics
        print(f"Running Ragas evaluation...")
        eval_start = time.time()
        
        result = evaluate(
            dataset=eval_dataset,
            metrics=[context_precision, context_recall, context_entity_recall]
        )
        
        eval_time = time.time() - eval_start
        
        # Extract metric scores from the result
        result_df = result.to_pandas()
        
        # Calculate mean scores for each metric
        precision_score = result_df['context_precision'].mean() if 'context_precision' in result_df.columns else 0
        recall_score = result_df['context_recall'].mean() if 'context_recall' in result_df.columns else 0
        entity_recall_score = result_df['context_entity_recall'].mean() if 'context_entity_recall' in result_df.columns else 0
        
        # Store results
        evaluation_results.append({
            "Retriever": retriever_name,
            "Context Precision": precision_score,
            "Context Recall": recall_score,
            "Context Entity Recall": entity_recall_score,
            "Avg Latency (ms)": round(avg_latency_ms, 2),
            "Total Retrieval Time (s)": round(retrieval_time, 2),
            "Evaluation Time (s)": round(eval_time, 2)
        })
        
        print(f"✅ Completed: {retriever_name}")
        print(f"   - Context Precision: {precision_score:.4f}")
        print(f"   - Context Recall: {recall_score:.4f}")
        print(f"   - Context Entity Recall: {entity_recall_score:.4f}")
        print(f"   - Avg Latency: {avg_latency_ms:.2f}ms")
        
    except Exception as e:
        print(f"❌ Error evaluating {retriever_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        evaluation_results.append({
            "Retriever": retriever_name,
            "Context Precision": 0,
            "Context Recall": 0,
            "Context Entity Recall": 0,
            "Avg Latency (ms)": 0,
            "Total Retrieval Time (s)": 0,
            "Evaluation Time (s)": 0,
            "Error": str(e)
        })

print("\n" + "="*60)
print("Evaluation complete!")
print("="*60)


### Step 5: Display Evaluation Results

Let's visualize the results in a comprehensive table.


In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(evaluation_results)

# Sort by a composite score (weighted average of metrics)
results_df['Composite Score'] = (
    0.35 * results_df['Context Precision'] + 
    0.35 * results_df['Context Recall'] + 
    0.30 * results_df['Context Entity Recall']
)

# Sort by composite score
results_df = results_df.sort_values('Composite Score', ascending=False)

# Display results
print("\n📊 RETRIEVER EVALUATION RESULTS")
print("="*100)
display(results_df)

# Create a focused view
print("\n🎯 Performance Ranking (by Composite Score):")
for idx, row in results_df.iterrows():
    print(f"{idx+1}. {row['Retriever']}: {row['Composite Score']:.4f}")


### Step 6: Analyze Cost Information from LangSmith

Let's fetch cost data from LangSmith to understand the financial implications of each retriever.


In [ ]:
# Fetch cost information from LangSmith
try:
    # Get runs from the current project
    runs = list(langsmith_client.list_runs(
        project_name="09_Advanced_Retrieval_Evaluation",
        limit=500
    ))
    
    if runs:
        total_cost = sum([run.total_cost for run in runs if run.total_cost])
        total_tokens = sum([run.total_tokens for run in runs if run.total_tokens])
        
        print(f"\n💰 COST ANALYSIS FROM LANGSMITH")
        print("="*60)
        print(f"Total Runs Tracked: {len(runs)}")
        print(f"Total Cost: ${total_cost:.4f}")
        print(f"Total Tokens Used: {total_tokens:,}")
        print(f"Average Cost per Run: ${total_cost/len(runs):.6f}")
        
        # Estimate cost per retriever (approximate)
        est_cost_per_retriever = total_cost / len(retrievers_to_evaluate)
        print(f"\nEstimated Cost per Retriever: ${est_cost_per_retriever:.4f}")
        
        # Add cost estimates to results
        results_df['Estimated Cost ($)'] = est_cost_per_retriever
        
    else:
        print("⚠️ No runs found in LangSmith. Make sure tracing is enabled.")
        print("You can view detailed cost and latency information in the LangSmith UI:")
        print("https://smith.langchain.com/")
        
except Exception as e:
    print(f"⚠️ Could not fetch cost data from LangSmith: {e}")
    print("Note: You can view detailed cost and latency information in the LangSmith UI:")
    print("https://smith.langchain.com/")


### Step 7: Visualize the Results

Let's create visualizations to better understand the trade-offs between retrievers.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create subplots for different metrics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Retriever Performance Comparison', fontsize=16, fontweight='bold')

# 1. Performance Metrics Comparison
ax1 = axes[0, 0]
metrics = ['Context Precision', 'Context Recall', 'Context Entity Recall']
x = np.arange(len(results_df))
width = 0.25

for i, metric in enumerate(metrics):
    ax1.bar(x + i*width, results_df[metric], width, label=metric, alpha=0.8)

ax1.set_xlabel('Retriever')
ax1.set_ylabel('Score')
ax1.set_title('Performance Metrics by Retriever')
ax1.set_xticks(x + width)
ax1.set_xticklabels(results_df['Retriever'], rotation=45, ha='right')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# 2. Composite Score
ax2 = axes[0, 1]
bars = ax2.barh(results_df['Retriever'], results_df['Composite Score'], color='skyblue')
ax2.set_xlabel('Composite Score')
ax2.set_title('Overall Performance Ranking')
ax2.grid(axis='x', alpha=0.3)

# Color the best performer
bars[0].set_color('gold')

# 3. Latency Comparison
ax3 = axes[1, 0]
ax3.bar(results_df['Retriever'], results_df['Avg Latency (ms)'], color='coral', alpha=0.7)
ax3.set_xlabel('Retriever')
ax3.set_ylabel('Average Latency (ms)')
ax3.set_title('Latency Comparison')
ax3.set_xticklabels(results_df['Retriever'], rotation=45, ha='right')
ax3.grid(axis='y', alpha=0.3)

# 4. Performance vs Latency Trade-off
ax4 = axes[1, 1]
scatter = ax4.scatter(results_df['Avg Latency (ms)'], 
                      results_df['Composite Score'], 
                      s=200, 
                      c=results_df['Composite Score'], 
                      cmap='viridis',
                      alpha=0.6,
                      edgecolors='black',
                      linewidth=2)

# Annotate points
for idx, row in results_df.iterrows():
    ax4.annotate(row['Retriever'], 
                (row['Avg Latency (ms)'], row['Composite Score']),
                fontsize=8,
                ha='right')

ax4.set_xlabel('Average Latency (ms)')
ax4.set_ylabel('Composite Score')
ax4.set_title('Performance vs Latency Trade-off')
ax4.grid(alpha=0.3)
plt.colorbar(scatter, ax=ax4, label='Composite Score')

plt.tight_layout()
plt.show()


### Step 8: Final Analysis and Recommendations

Based on the evaluation results, let's provide a comprehensive analysis considering performance, cost, and latency.


In [ ]:
print("\n" + "="*100)
print("📝 COMPREHENSIVE ANALYSIS AND RECOMMENDATIONS")
print("="*100 + "\n")

# Get the best performers in different categories
best_overall = results_df.iloc[0]
fastest = results_df.loc[results_df['Avg Latency (ms)'].idxmin()]
best_precision = results_df.loc[results_df['Context Precision'].idxmax()]
best_recall = results_df.loc[results_df['Context Recall'].idxmax()]

analysis = f"""
EXECUTIVE SUMMARY:

After evaluating {len(retrievers_to_evaluate)} different retrieval methods on our synthetic usecase dataset, 
here are the key findings:

🏆 BEST OVERALL PERFORMER: {best_overall['Retriever']}
   - Composite Score: {best_overall['Composite Score']:.4f}
   - Context Precision: {best_overall['Context Precision']:.4f}
   - Context Recall: {best_overall['Context Recall']:.4f}
   - Context Entity Recall: {best_overall['Context Entity Recall']:.4f}
   - Average Latency: {best_overall['Avg Latency (ms)']:.2f}ms

⚡ FASTEST RETRIEVER: {fastest['Retriever']}
   - Average Latency: {fastest['Avg Latency (ms)']:.2f}ms
   - Composite Score: {fastest['Composite Score']:.4f}

🎯 BEST PRECISION: {best_precision['Retriever']}
   - Context Precision: {best_precision['Context Precision']:.4f}

📚 BEST RECALL: {best_recall['Retriever']}
   - Context Recall: {best_recall['Context Recall']:.4f}

DETAILED ANALYSIS:

For this particular synthetic usecase dataset about AI projects with various domains, the evaluation 
reveals important trade-offs between different retrieval strategies:

1. PERFORMANCE ANALYSIS:
   {best_overall['Retriever']} emerged as the top performer with a composite score of {best_overall['Composite Score']:.4f}.
   This retriever demonstrates {
       'strong' if best_overall['Context Precision'] > 0.7 else 
       'moderate' if best_overall['Context Precision'] > 0.5 else 'limited'
   } precision ({best_overall['Context Precision']:.4f}) and {
       'excellent' if best_overall['Context Recall'] > 0.7 else 
       'good' if best_overall['Context Recall'] > 0.5 else 'adequate'
   } recall ({best_overall['Context Recall']:.4f}), making it highly effective at retrieving 
   relevant project information while minimizing irrelevant results.

2. LATENCY CONSIDERATIONS:
   The fastest retriever, {fastest['Retriever']}, processes queries in just {fastest['Avg Latency (ms)']:.2f}ms
   compared to {results_df['Avg Latency (ms)'].max():.2f}ms for the slowest. For production systems
   requiring real-time responses, {'this makes it an excellent choice despite potentially lower accuracy'
   if fastest['Retriever'] != best_overall['Retriever'] else 'its combination of speed and accuracy is ideal'}.

3. COST EFFICIENCY:
   Based on LangSmith tracking, retrieval operations have measurable costs through API calls to 
   embedding models and LLMs (for multi-query and evaluation). Methods like BM25 that rely on 
   lexical matching have zero ongoing API costs, while reranking and multi-query approaches 
   incur additional expenses through multiple model calls.

4. USE CASE RECOMMENDATIONS:
   
   - For PRODUCTION SYSTEMS with strict latency requirements (<100ms): 
     Use {fastest['Retriever']} or implement caching strategies
   
   - For ACCURACY-CRITICAL APPLICATIONS (research, compliance):
     Use {best_overall['Retriever']} to maximize retrieval quality
   
   - For COST-SENSITIVE DEPLOYMENTS:
     Start with BM25 or Naive retrieval, then add complexity only if needed
   
   - For BALANCED APPROACH:
     Ensemble methods can provide good performance by combining multiple strategies,
     though they increase latency and cost proportionally

CONCLUSION:
{
f'''The {best_overall['Retriever']} retriever is recommended for this dataset because it achieves 
the best balance of precision and recall, ensuring that users receive comprehensive and relevant 
project information. While it may have {'higher latency than simpler methods' if best_overall['Avg Latency (ms)'] > fastest['Avg Latency (ms)'] else 'excellent latency characteristics'}, 
the improved accuracy justifies the tradeoff for most use cases. Organizations should consider 
implementing caching, async processing, or tiered retrieval strategies to optimize the cost-performance-latency 
triangle based on their specific requirements.'''
}

For detailed cost breakdowns and trace-level latency analysis, refer to the LangSmith dashboard at:
https://smith.langchain.com/projects/09_Advanced_Retrieval_Evaluation
"""

print(analysis)

# Save results to CSV for further analysis
results_df.to_csv('retriever_evaluation_results.csv', index=False)
print("\n✅ Results saved to 'retriever_evaluation_results.csv'")


### 🎯 Key Takeaways

**Summary of Retriever Evaluation:**

This comprehensive evaluation demonstrates that choosing the right retrieval strategy depends heavily on your specific requirements:

1. **No One-Size-Fits-All Solution**: Each retriever has distinct strengths and weaknesses across performance, cost, and latency dimensions.

2. **The Performance-Latency Trade-off**: More sophisticated retrievers (Multi-Query, Contextual Compression, Ensemble) typically achieve better accuracy but at the cost of increased latency and API calls.

3. **Ensemble Benefits**: Ensemble retrievers can leverage the strengths of multiple approaches, though they compound latency and costs.

4. **Cost Considerations**: 
   - BM25: Zero ongoing API costs (pure keyword matching)
   - Naive/Parent Document: Minimal costs (single embedding call)
   - Multi-Query: 3-5x embedding costs (multiple queries)
   - Contextual Compression: Additional reranking API costs
   - Ensemble: Combined costs of all constituent retrievers

5. **Production Recommendations**:
   - **Start simple**: Begin with Naive or BM25, measure baseline performance
   - **Iterate intelligently**: Add complexity only where needed based on actual performance gaps
   - **Monitor continuously**: Use LangSmith to track costs and latency in production
   - **Consider caching**: Cache frequent queries to amortize retrieval costs
   - **Hybrid approaches**: Combine cheap BM25 for initial filtering with expensive reranking for top candidates

6. **Dataset-Specific Insights**: 
   For structured project data with clear domains and descriptions, the retrieval strategy should align with query patterns:
   - Exact term matches → BM25
   - Semantic similarity → Vector embeddings
   - Complex queries → Multi-Query or Ensemble
   - Maximum accuracy → Contextual Compression (Rerank)

**Next Steps**: 
- Review the LangSmith dashboard for detailed trace analysis
- Experiment with different retriever configurations
- Test with real user queries to validate synthetic evaluation results
- Implement A/B testing in production to measure real-world impact
